In [1]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("External Data")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '4g')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/08/19 06:38:19 WARN Utils: Your hostname, flgg resolves to a loopback address: 127.0.1.1; using 172.28.210.36 instead (on interface eth0)
23/08/19 06:38:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/19 06:38:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/19 06:38:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
import pandas as pd

In [4]:
csv = pd.read_csv('../data/raw/NYPD_crime_data.csv')

In [5]:
csv.count()

CMPLNT_NUM           271717
ADDR_PCT_CD          271683
BORO_NM              271717
CMPLNT_FR_DT         271717
CMPLNT_FR_TM         271717
CMPLNT_TO_DT         253642
CMPLNT_TO_TM         271717
CRM_ATPT_CPTD_CD     271717
HADEVELOPT           271717
JURIS_DESC           271717
KY_CD                271717
LAW_CAT_CD           271717
LOC_OF_OCCUR_DESC    271717
OFNS_DESC            271717
PARKS_NM             271717
PD_CD                271516
PD_DESC              271717
PREM_TYP_DESC        271717
RPT_DT               271717
Lat_Lon              271712
X_COORD_CD           271712
Y_COORD_CD           271712
Latitude             271712
Longitude            271712
dtype: int64

In [6]:
# filter the train data and test data
csv['CMPLNT_FR_DT'] = pd.to_datetime(csv['CMPLNT_FR_DT'], format='%m/%d/%Y', errors='coerce')

start_date = '2022-09-01'
end_date = '2023-03-31'

filtered_df = csv[(csv['CMPLNT_FR_DT'] >= start_date) & (csv['CMPLNT_FR_DT'] <= end_date)]


test_start_date = '2023-04-01'
test_end_date = '2023-04-30'

test_cdf = csv[(csv['CMPLNT_FR_DT'] >= test_start_date) & (csv['CMPLNT_FR_DT'] <= test_end_date)]

In [7]:
# dropna
cols_to_check = ['ADDR_PCT_CD', 'BORO_NM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'CMPLNT_TO_DT', 'CMPLNT_TO_TM']

filtered_df = filtered_df.dropna(subset=cols_to_check)

test_cdf = test_cdf.dropna(subset=cols_to_check)

filtered_df.count()
test_cdf.count()


CMPLNT_NUM           40562
ADDR_PCT_CD          40562
BORO_NM              40562
CMPLNT_FR_DT         40562
CMPLNT_FR_TM         40562
CMPLNT_TO_DT         40562
CMPLNT_TO_TM         40562
CRM_ATPT_CPTD_CD     40562
HADEVELOPT           40562
JURIS_DESC           40562
KY_CD                40562
LAW_CAT_CD           40562
LOC_OF_OCCUR_DESC    40562
OFNS_DESC            40562
PARKS_NM             40562
PD_CD                40562
PD_DESC              40562
PREM_TYP_DESC        40562
RPT_DT               40562
Lat_Lon              40562
X_COORD_CD           40562
Y_COORD_CD           40562
Latitude             40562
Longitude            40562
dtype: int64

In [9]:
# read in the geo data for merge
import pandas as pd
import geopandas as gpd

sf = gpd.read_file("../data/landing/taxi_zones.shp")
zones = pd.read_csv("../data/landing/taxi+_zone_lookup.csv")
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
sf.head()


gdf = gpd.GeoDataFrame(
    pd.merge(zones, sf, on='LocationID', how='inner')
)

In [10]:
# combined the data according to the taxi zone data
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(filtered_df.Longitude, filtered_df.Latitude)]
points_gdf = gpd.GeoDataFrame(filtered_df, geometry=geometry)

test_geometry = [Point(xy) for xy in zip(test_cdf.Longitude, test_cdf.Latitude)]
test_points_gdf = gpd.GeoDataFrame(test_cdf, geometry=test_geometry)


In [12]:
# generate and save the geo train and test data
joined = gpd.sjoin(points_gdf, sf, how="inner", op="within")
joined.to_csv('../data/curated/joined.csv', index=False)

test_joined = gpd.sjoin(test_points_gdf, sf, how="inner", op="within")
test_joined.to_csv('../data/curated/test_joined.csv', index=False)


/home/flgg/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_363131/204773563.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs + ...

  joined = gpd.sjoin(points_gdf, sf, how="inner", op="within")
/home/flgg/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_363131/204773563.py:4: UserWarning: CRS mismatch between the CRS of left g